In [50]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
import warnings
warnings.filterwarnings("ignore")

In [52]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,5)
plt.rcParams['axes.grid'] = False
import seaborn as sns
sns.set_style("whitegrid", {'axes.grid' : False})


In [53]:
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,5)
plt.rcParams['axes.grid'] = False
import seaborn as sns
sns.set_style("whitegrid", {'axes.grid' : False})

import numpy as np
import itertools
import pandas as pd
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import InputLayer
from keras.layers import Dense
import neptune.new as neptune
from datetime import datetime

from Utils.add_features import add_fisher
from Data.data_retrieval import get_data
from Utils.neptune_ai_api_key import API_KEY

np.random.seed(12)

In [193]:
def get_stock_data(symbol):
    if symbol == 'sinx':
        df = get_data(".NSEI", 'D')
        df.drop(columns=["Volume"], inplace=True)
        df["Close"] = df["Open"] = df["High"] = df["Low"] = np.sin(df.index / 10 ) +2
    else:
        df = get_data(symbol, 'D')
    df.set_index("Datetime", inplace=True)
    df.dropna(inplace=True)
    return df

def f_discretize(values, num_states=10):
    states_value = dict()
    step_size = 1./num_states
    for i in range(num_states):
        if i == num_states - 1:
            states_value[i] = values.max()
        else:
            states_value[i] = values.quantile((i+1)*step_size)
    return states_value

def value_to_state(value, states_value):
    if np.isnan(value):
        return np.nan
    else:
        for state, v in states_value.items():
            if value <= v:
                return str(state)
        return str(state)

def add_features(df, features):
    lookbacks = []
    all_states = []
    for feature, lookback, discretize in [(feature["feature"], feature["lookback"], feature["discretize"]) for feature in features]:
        lookbacks.append(lookback)
        if feature == "Close":
            if discretize>0:
                states = f_discretize(df["Close"].iloc[lookback:int(df.shape[0]*0.8)], discretize)
                df[f"{feature}_state"] = df['Close'].apply(lambda x : value_to_state(x, states))
        if feature.startswith("Fisher"):
            df[feature] = add_fisher([df, lookback])[[f"Fisher{lookback}"]]
            if discretize>0:
                states = f_discretize(df[feature].iloc[lookback:int(df.shape[0]*0.8)], discretize)
                df[f"{feature}_state"] = df[feature].apply(lambda x : value_to_state(x, states))
        if feature.startswith("Momentum"):
            df[feature] = df["Close"].diff(lookback)
            if discretize>0:
                states = f_discretize(df[feature].iloc[lookback:int(df.shape[0]*0.8)], discretize)
                df[f"{feature}_state"] = df[feature].apply(lambda x : value_to_state(x, states))
        try:
            all_states.append({'Feature': feature, 'states':states})
        except:
            pass

    df = df.iloc[max(lookbacks):]
    return df, all_states

def initialize_q_mat(all_states, all_actions):
    states_size = len(all_states)
    actions_size = len(all_actions)
    q_mat = np.random.rand(states_size, actions_size) / 1e9
    q_mat = pd.DataFrame(q_mat, columns=all_actions.keys())
    q_mat['states'] = all_states
    q_mat.set_index('states', inplace=True)
    return q_mat

def train_q_learning(train, state_lookback, q, alpha, epsilon, gamma, episodes, metric="absolute" , theta=2):
    train_df = train.copy()
    train_df["PctReturns"] = train_df["Close"]/train_df["Close"].shift(1)-1
    train_df["AbsReturns"] = train_df["Close"].diff(1)
    train_df.dropna(inplace=True)
    dates = list(train_df.index)
    train_data = train_df[["Close", "state"]]
    actions_history = []
    returns_since_entry = [0]
    train_returns = []
    equity_curve = []

    for ii in tqdm(range(episodes)):
        actions_history = []
        num_shares = 0
        bought_history = []
        bought_history_dates = []
        returns_since_entry = [0]
        equity_curve = []

        for i, val in enumerate(np.array(train_data)):
            if i<state_lookback:
                continue
            current_adj_close, state = val
            prev_adj_close, _ = np.array(train_data)[i - 1]
            fisher_state = train_df.iloc[i]["Fisher_state"]
            state = ''.join(list(train_data.iloc[i - state_lookback:i]["state"])[::-1])
            current_date = dates[i]
            try:
                equity_curve.append(equity_curve[-1]+num_shares * (current_adj_close-prev_adj_close))
            except:
                equity_curve.append(0)

            if len(bought_history) > 0:
                returns_since_entry.append(get_return_since_entry(bought_history, current_adj_close))
            else:
                returns_since_entry.append(returns_since_entry[-1])

            # decide action
            if epsilon > 0.1:
                epsilon = epsilon / (i + 1)

            if use_nn:
                action_priority = act_nn(state, model, num_states, threshold=epsilon, actions_size=3)
            else:
                action_priority = act(state, q, threshold=epsilon, actions_size=3)

            action = action_priority[0]

            # Constraining max number of units bought
            if (num_shares >= max_units) and (action == 1):
                action = action_priority[1]

            # Short Selling not Allowed
            if (num_shares <= 0) and (action == 2):
                action = action_priority[1]

            # Forcing Trades
            # if state[:state_lookback]=='0'*state_lookback:
            #     action=1
            # elif state[:state_lookback]=='9'*state_lookback:
            #     if num_shares==0:
            #         action=0
            #     else:
            #         action=2
            # else:
            #     action=0
            #
            # # Constraining max number of units bought
            # if (num_shares >= max_units) and (action == 1):
            #     action = 0
            #
            # # Short Selling not Allowed
            # if (num_shares <= 0) and (action == 2):
            #     action = 0

            # get reward
            if action == 0:  # hold
                if num_shares > 0:
                    past = current_adj_close - prev_adj_close   #should this be multiplied by num_shares
                    if metric == "percent":
                        reward = past/prev_adj_close
                    else:
                        reward = past
                else:
                    reward = 0

            if action == 1:  # buy
                if fisher_state=='0':
                    reward = buy_reward
                else:
                    reward = 0
                num_shares += 1
                bought_history.append((current_adj_close))
                bought_history_dates.append((current_date))

            if action == 2:  # sell
                bought_date = bought_history_dates[0]
                #penalize negative moves
                if metric=="percent":
                    reward = get_sell_reward(list(train_df.loc[bought_date:current_date]["PctReturns"]), theta, type=metric)
                if metric == "absolute":
                    reward = get_sell_reward(list(train_df.loc[bought_date:current_date]["AbsReturns"]), theta, type=metric)
                bought_history.pop(0)
                bought_history_dates.pop(0)
                num_shares -= 1

            try:
                next_adj_close, next_state = np.array(train_data)[i + 1]
                next_state = ''.join(list(train_data.iloc[i - state_lookback+1:i+1]["state"])[::-1])
            except:
                break


            # print(f"Train action: {action}")
            # print(f"State: {state}")
            # print(f"Reward: {reward}")
            # print(f"Num units: {num_shares}")
            # print("*"*100)



            actions_history.append((i, current_adj_close, action))

            if use_nn:
                target = reward + gamma *np.max(model.predict(np.identity(num_states)[int(next_state):int(next_state) + 1]))
                target_vector = model.predict(np.identity(num_states)[int(state):int(state) + 1])[0]
                target_vector[action] = target
                model.fit(np.identity(num_states)[int(state):int(state) + 1], target_vector.reshape(-1, 3),epochs=1, verbose=0)
            else:
                # update q table
                q.loc[state, action] = (1. - alpha) * q.loc[state, action] + alpha * (
                            reward + gamma * (q.loc[next_state].max()))

        train_returns.append(get_invested_capital(actions_history, returns_since_entry))


    train_results_over_episodes = train_returns
    print('End of Training!')
    return q, actions_history, returns_since_entry, model, train_results_over_episodes, equity_curve


# Data Prep

Get data

In [222]:
df = get_stock_data('sinx')
train_len = int(df.shape[0]*0.8)

Add Features

In [223]:
features = [
        {"feature": "Close", "lookback": 0, "discretize": 10},
        {"feature": "Momentum", "lookback": 1, "discretize": 2}
    ]

In [224]:
df, all_states = add_features(df, features)

In [225]:
df["state"] = df[[f"{feature['feature']}_state" for feature in features]].sum(axis=1).astype(int).astype(str)

In [226]:
train_df = df.iloc[:train_len, :]
test_df = df.iloc[train_len:, :]

# QLearning

Define all actions

In [227]:
all_actions = {0: 'neutral', 1: 'long'}

Get All states

In [228]:
states = [list(state['states']) for state in all_states]
all_states = list(itertools.product(*states))
all_states = [''.join(tuple([str(state) for state in all_state])) for all_state in all_states]

Initialise qmatrix with random vals

In [229]:
q = initialize_q_mat(all_states, all_actions) / 1e9

In [ ]:
use_nn = False
alpha = 0.1
epsilon = 0.9
gamma = 0.9
episodes = 10
max_units = 1
metric = "percent"
theta = 1
buy_reward = 0
q, train_actions_history, train_returns_since_entry, _, train_results_over_episodes, train_equity_curve = train_q_learning(
    train_data, state_lookback, q, states_size, use_nn=use_nn, alpha=alpha, epsilon=epsilon, gamma=gamma,
    episodes=episodes, buy_reward=buy_reward, max_units=max_units, metric=metric, theta=theta)